In [1]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_colwidth',1000)

In [ ]:
from src.utils import get_city_lat_long

# Load the dataset
data_path = os.path.join(os.path.abspath(os.path.join(os.getcwd(), os.pardir)), "artifacts", "population.csv")
df = pd.read_csv(data_path)

# Remove all rows that don't have 'city' in the City column
df = df.loc[df['City'].str.contains('city', case=False), :]

# Remove the word 'city' and 'balance' from the City column
df['City'] = df['City'].str.replace(' city', '', case=False)
df['City'] = df['City'].str.replace(' (balance)', '', case=False)

# Add latitude and longitude columns to the dataframe
df['Latitude'] = None
df['Longitude'] = None

# Iterate over the rows to get latitude and longitude
for index, row in df.iterrows():
    city = row['City']
    state = row['State']
    lat, long = get_city_lat_long(city, state)
    df.at[index, 'Latitude'] = lat
    df.at[index, 'Longitude'] = long

df.head()

,City,State,Population,Latitude,Longitude
0,New York,New York,8258035,40.712775,-74.005973
1,Los Angeles,California,3820914,34.054908,-118.242643
2,Chicago,Illinois,2664452,41.878114,-87.629798
3,Houston,Texas,2314157,29.760077,-95.370111
4,Phoenix,Arizona,1650070,33.448377,-112.074037


In [ ]:
df['volume (ton)'] = (df['Population'] * 1.00) / 1000 # For small IT waste. Assuming ~1 kg per capita per year for small IT waste
df.head()

,City,State,Population,Latitude,Longitude,volume (ton)
0,New York,New York,8258035,40.712775,-74.005973,8258.035
1,Los Angeles,California,3820914,34.054908,-118.242643,3820.914
2,Chicago,Illinois,2664452,41.878114,-87.629798,2664.452
3,Houston,Texas,2314157,29.760077,-95.370111,2314.157
4,Phoenix,Arizona,1650070,33.448377,-112.074037,1650.070


In [ ]:
df.drop(["Population"], axis = 1, inplace=True)
df.head()

,City,State,Latitude,Longitude,volume (ton)
0,New York,New York,40.712775,-74.005973,8258.035
1,Los Angeles,California,34.054908,-118.242643,3820.914
2,Chicago,Illinois,41.878114,-87.629798,2664.452
3,Houston,Texas,29.760077,-95.370111,2314.157
4,Phoenix,Arizona,33.448377,-112.074037,1650.070


In [ ]:
code = ["W" + item for item in np.arange(1, df.shape[0]+1).astype(str)]
df['Code'] = code
df.head()

,City,State,Latitude,Longitude,volume (ton),Code
0,New York,New York,40.712775,-74.005973,8258.035,W1
1,Los Angeles,California,34.054908,-118.242643,3820.914,W2
2,Chicago,Illinois,41.878114,-87.629798,2664.452,W3
3,Houston,Texas,29.760077,-95.370111,2314.157,W4
4,Phoenix,Arizona,33.448377,-112.074037,1650.070,W5


In [ ]:
from src.utils import add_sheet_to_excelbook

data_path = os.path.join(os.path.abspath(os.path.join(os.getcwd(), os.pardir)), "artifacts", "input_data.xlsx")
waste_volume = df[['Code', 'volume (ton)']]
add_sheet_to_excelbook(data_path, "Waste Volume", waste_volume)

waste_source = df[['Code', 'City', 'State', 'Latitude', 'Longitude']]
add_sheet_to_excelbook(data_path, "Waste Source", waste_source)

In [2]:
data_path = os.path.join(os.path.abspath(os.path.join(os.getcwd(), os.pardir)), "artifacts", "input_data.xlsx")
waste_source_data = pd.read_excel(data_path, sheet_name='Waste Source')
shredder_data = pd.read_excel(data_path, sheet_name='E-waste Collector')
oxide_producer_data = pd.read_excel(data_path, sheet_name='Oxide Producer')
fluoride_producer_data = pd.read_excel(data_path, sheet_name='Flouride Producer')
metal_producer_data = pd.read_excel(data_path, sheet_name='Metal Producer')
magnet_producer_data = pd.read_excel(data_path, sheet_name='Magnet Producer')

# Create dictionaries for shredder and oxide producer coordinates
waste_source = {row['Code']: (row['Latitude'], row['Longitude']) for index, row in waste_source_data.iterrows()}
shredders = {row['Code']: (row['Latitude'], row['Longitude']) for index, row in shredder_data.iterrows()}
oxide_producers = {row['Code']: (row['Latitude'], row['Longitude']) for index, row in oxide_producer_data.iterrows()}
fluoride_producers = {row['Code']: (row['Latitude'], row['Longitude']) for index, row in fluoride_producer_data.iterrows()}
metal_producers = {row['Code']: (row['Latitude'], row['Longitude']) for index, row in metal_producer_data.iterrows()}
magnet_producers = {row['Code']: (row['Latitude'], row['Longitude']) for index, row in magnet_producer_data.iterrows()}

In [3]:
from src.utils import haversine
cost_per_mile_per_ton = 0.36

cost_data = []
for loc1 in waste_source:
    row = []
    lat1, lon1 = waste_source[loc1]
    
    for loc2 in shredders:
        lat2, lon2 = shredders[loc2]
        distance = haversine(lat1, lon1, lat2, lon2)
        row.append(distance * cost_per_mile_per_ton)
    
    cost_data.append(row)

Waste_to_shredder_tc = pd.DataFrame(cost_data, index=waste_source.keys(), columns=shredders.keys())
Waste_to_shredder_tc.index.name = 'index'
Waste_to_shredder_tc.reset_index(inplace=True, drop=False)

In [4]:
Waste_to_shredder_tc.head()

,index,S01,S02,S03,S04,S05,S06,S07,S08,S09,...,S751,S752,S753,S754,S755,S756,S757,S758,S759,S760
0,W1,230.710432,225.867550,235.560713,242.944894,312.440701,207.941734,502.274402,200.260080,1.281750,...,871.827529,831.528677,880.346014,787.376706,868.709342,862.503785,866.213496,825.475077,864.958343,787.428104
1,W2,651.299245,659.808774,646.678325,747.665595,572.195562,672.842294,435.039881,683.492862,880.712617,...,369.079550,307.200825,341.147684,336.361559,335.665536,368.427916,336.293087,343.103172,346.772445,369.454375
2,W3,59.633613,80.598591,59.667128,241.216543,92.016206,52.361805,295.316424,85.043896,256.316148,...,633.747139,586.578488,638.487532,546.051791,626.489190,624.607170,624.124033,584.920942,624.176645,550.897509
3,W4,312.664866,304.562898,308.042250,297.441243,247.272493,349.310927,86.476667,329.450572,509.949319,...,699.611229,631.520579,688.346217,615.502269,676.810101,692.704403,675.421826,649.777544,681.004509,637.711816
4,W5,539.862646,546.272356,535.042362,623.644749,458.673722,564.674077,307.839579,571.001416,770.552776,...,424.668992,354.692632,401.994272,366.716397,393.390994,421.290951,393.165992,386.207061,402.298012,398.761904


In [5]:
cost_data = []
for loc1 in shredders:
    row = []
    lat1, lon1 = shredders[loc1]
    
    for loc2 in oxide_producers:
        lat2, lon2 = oxide_producers[loc2]
        distance = haversine(lat1, lon1, lat2, lon2)
        row.append(distance * cost_per_mile_per_ton)
    
    cost_data.append(row)

shredder_to_oxide_tc = pd.DataFrame(cost_data, index=shredders.keys(), columns=oxide_producers.keys())
shredder_to_oxide_tc.index.name = 'index'
shredder_to_oxide_tc.reset_index(inplace=True, drop=False)

In [6]:
shredder_to_oxide_tc.head()

,index,OP01,OP02,OP03,OP04,OP05,OP06,OP07,OP08,OP09,...,OP56,OP57,OP58,OP59,OP60,OP61,OP62,OP63,OP64,OP65
0,S01,334.407348,488.348682,564.060175,485.621957,90.927425,183.035852,275.984457,270.420928,277.394211,...,374.883217,1.098491,17.322992,57.706791,531.452322,79.657983,696.861825,264.607448,159.607079,156.414433
1,S02,329.271919,501.179926,579.118630,498.013921,74.482482,183.409096,273.018561,267.698212,274.655395,...,375.247801,21.249216,17.778701,73.857514,548.587929,98.201676,708.259641,267.194946,142.142945,172.878489
2,S03,329.528755,484.346550,560.552164,481.538696,88.137240,177.980781,270.998559,265.426105,272.399187,...,369.826365,3.995813,13.493944,54.236763,528.557834,82.448202,692.624648,269.651482,162.253392,153.220321
3,S04,341.867587,619.530380,708.982162,613.881510,121.805879,258.872542,309.576508,306.807535,312.495259,...,422.145286,181.878228,172.800296,227.173964,690.522340,251.509475,813.836572,325.497049,86.897086,319.839663
4,S05,261.348719,417.930153,500.711482,414.022882,92.080114,103.153579,199.567647,193.667781,200.576604,...,293.797556,81.191842,70.811249,49.120494,477.815578,135.130902,622.594985,345.281700,221.330893,108.960581


In [ ]:
from src.utils import calculate_transportation_cost
import time

#waste_to_shredder_tc = calculate_transportation_cost(waste_source, shredders)
#time.sleep(5.5)
#shredder_to_oxide_tc = calculate_transportation_cost(shredders, oxide_producers)
#time.sleep(5.5)
oxide_to_fluoride_tc = calculate_transportation_cost(oxide_producers, fluoride_producers)
time.sleep(5.5)
fluoride_to_metal_tc = calculate_transportation_cost(fluoride_producers, metal_producers)
time.sleep(5.5)
metal_to_magnet_tc = calculate_transportation_cost(metal_producers, magnet_producers)
time.sleep(5.5)


In [ ]:
from src.utils import add_sheet_to_excelbook
add_sheet_to_excelbook(data_path, "Shredder-Oxide TC", shredder_to_oxide_tc)
add_sheet_to_excelbook(data_path, "Oxide-Fluoride TC", oxide_to_fluoride_tc)
add_sheet_to_excelbook(data_path, "Fluoride-Metal TC", fluoride_to_metal_tc)
add_sheet_to_excelbook(data_path, "Metal-Magnet TC", metal_to_magnet_tc)
add_sheet_to_excelbook(data_path, "Waste-Shredder TC", Waste_to_shredder_tc)